In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
%matplotlib notebook
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sklearn
from sklearn import model_selection, linear_model, svm, naive_bayes, neighbors, ensemble, metrics, datasets
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage.color import *
import cv2
import os
import glob
import mahotas as mt
import random
import pickle
import gc
from PIL import Image
import lightgbm as lgb
from xgboost import XGBClassifier
seed = random.seed(42)

In [22]:
# Load images
class1 = 'Clean_Room'
class2 = 'Messy_Room'
class3 = 'Bad_Excuse'
path_data = '/Users/regina/Desktop/Metis/Metis Projects/miSPICKy/dataset/'
images_class1 = ['/Users/regina/Desktop/Metis/Metis Projects/miSPICKy/dataset/Clean_Room/{}'.format(i) for i in os.listdir(path_data+str(class1)) if str(class1) in i]
images_class2 = ['/Users/regina/Desktop/Metis/Metis Projects/miSPICKy/dataset/Messy_Room/{}'.format(i) for i in os.listdir(path_data+str(class2)) if str(class2) in i]
images_class3 = ['/Users/regina/Desktop/Metis/Metis Projects/miSPICKy/Bad_Excuse/{}'.format(i) for i in os.listdir(path_data+str(class2)) if str(class2) in i]
# path_data = '/Users/shiheping/Documents/data_science_study/kaplan_metis_bootcamp/projects/project_3/artwork_images/image_training_classes/'
# images_class1 = ['/Users/shiheping/Documents/data_science_study/kaplan_metis_bootcamp/projects/project_3/artwork_images/image_training_classes/symbolism/{}'.format(i) for i in os.listdir(path_data+str(class1)) if str(class1) in i]
# images_class2 = ['/Users/shiheping/Documents/data_science_study/kaplan_metis_bootcamp/projects/project_3/artwork_images/image_training_classes/baroque/{}'.format(i) for i in os.listdir(path_data+str(class2)) if str(class2) in i]
train_class1 = images_class1[:500]
train_class2 = images_class2[:500]
train_class3 = images_class3[:500]
test_images = images_class1[500:600] + images_class2[500:600]+ images_class3[500:600]
train_images = train_class1 + train_class2 + train_class3
random.shuffle(train_images)
random.shuffle(test_images)
# clear useless list
del images_class1
del images_class2
del images_class3
gc.collect()

75

In [26]:
nrows = 100
ncolumns = 100
channels = 3

def create_features(img):
    # read image
    #open_image = cv2.imread(img,cv2.IMREAD_COLOR)
    open_image = cv2.resize(cv2.imread(img,cv2.IMREAD_COLOR),(nrows,ncolumns),interpolation=cv2.INTER_CUBIC)
    # convert to array
    img_to_array = np.array(open_image)
    # flatten three channel color image
    color_features = img_to_array.flatten()
    # convert image to greyscale
    grey_image = rgb2grey(img_to_array)
    # get HOG features from greyscale image
    hog_features = hog(grey_image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    # get Haralick (GLCM)
    grayscale = cv2.cvtColor(open_image,cv2.COLOR_BGR2GRAY)
    textures = mt.features.haralick(grayscale)
    ht_mean = textures.mean(axis=0)
    # combine color, hog features, Haralick into a single array
    flat_features = np.hstack((color_features,hog_features,ht_mean))
    return flat_features
# Haralick Texture is used to quantify an image based on texture. The fundamental
# concept involved in computing Haralick Texture features is the Gray Level Co-occurrence Matrix or GLCM.

# Loop through images to preprocess
def read_and_create_feature_matrix(list_of_images):
    """
    Returns two arrays:
    X is an array of resized images
    y is an array of labels
    """
    X = []
    y = []
    for image in list_of_images:
        image_features = create_features(image)
        X.append(image_features)
        if class1 in image:
            y.append(1)
        elif class2 in image:
            y.append(0)
    return np.array(X), np.array(y)

In [ ]:
nrows = 100
ncolumns = 100
channels = 3

def create_features(img):
    # read image
    #open_image = cv2.imread(img,cv2.IMREAD_COLOR)
    open_image = cv2.resize(cv2.imread(img,cv2.IMREAD_COLOR),(nrows,ncolumns),interpolation=cv2.INTER_CUBIC)
    # convert to array
    img_to_array = np.array(open_image)
    # flatten three channel color image
    color_features = img_to_array.flatten()
    # convert image to greyscale
    grey_image = rgb2grey(img_to_array)
    # get HOG features from greyscale image
    hog_features = hog(grey_image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    # get Haralick (GLCM)
    grayscale = cv2.cvtColor(open_image,cv2.COLOR_BGR2GRAY)
    textures = mt.features.haralick(grayscale)
    ht_mean = textures.mean(axis=0)
    # combine color, hog features, Haralick into a single array
    flat_features = np.hstack((color_features,hog_features,ht_mean))
    return flat_features
# Haralick Texture is used to quantify an image based on texture. The fundamental
# concept involved in computing Haralick Texture features is the Gray Level Co-occurrence Matrix or GLCM.

# Loop through images to preprocess
def read_and_create_feature_matrix(list_of_images):
    """
    Returns two arrays:
    X is an array of resized images
    y is an array of labels
    """
    X = []
    y = []
    for image in list_of_images:
        image_features = create_features(image)
        X.append(image_features)
        if class1 in image:
            y.append(1)
        elif class2 in image:
            y.append(0)
    return np.array(X), np.array(y)

In [28]:
display(Image.open(train_images[0]))

IndexError: list index out of range

In [25]:
print(train_images[0])

IndexError: list index out of range

In [12]:
# run create_feature_matrix on our dataframe of images
X, y = read_and_create_feature_matrix(train_images)
X_test, y_test = read_and_create_feature_matrix(test_images)
# Scale feature matrix + PCA
# get shape of feature matrix
print('Feature matrix shape is: ', X.shape)

Feature matrix shape is:  (0,)


In [13]:
# check function output
print('Feature matrix shape is: ', y.shape)
type(create_features(train_images[0]))
A = []
for i in range(3):
    a = np.array(create_features(train_images[i]))
    A.append(a)
print(np.array(A).shape)
print(np.array(cv2.imread(train_images[4],cv2.IMREAD_COLOR)).shape)
print(create_features(train_images[0]).shape)

Feature matrix shape is:  (0,)


IndexError: list index out of range

In [ ]:
# define standard scaler
ss = StandardScaler()
# run this on our feature matrix
X_scaled = ss.fit_transform(X)
X_test_scaled = ss.fit_transform(X_test)

pca = PCA(n_components=200)
# use fit_transform to run PCA on our standardized matrix
X_pca = pca.fit_transform(X_scaled)
X_test_pca = pca.fit_transform(X_test_scaled)
# look at new shape
print('PCA matrix shape is: ', X_pca.shape)
print('PCA matrix shape is: ', X_test_pca.shape)

In [ ]:
def train_cross_validate_evaluate(X_train, y_train, X_test, y_test):
    # instantiate classifier
    svc = svm.SVC()
    rfc = RandomForestClassifier(random_state=seed)
    knn = KNeighborsClassifier()
    gnb = GaussianNB()
    xgb = XGBClassifier()
    # GridsearchCV parameters
    param_grid_svm = [
      {'C': [1, 2, 5, 10, 100], 'kernel': ['linear']},
      {'C': [1, 2, 5, 10, 100], 'gamma': [0.5, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']},
     {'C': [1, 2, 5, 10, 100], 'gamma': [0.5, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['sigmoid']}]
    param_grid_rfc = {
    'n_estimators': [100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8,10],
    'criterion' :['gini', 'entropy']}
    param_grid_knn = {
    'n_neighbors': [3,5,11,19],
    'weights':['uniform','distance'],
    'metric':['euclidean','manhattan']}
    param_grid_xgb = {
    'reg_alpha': [0.0, 1.0], 
    'reg_lambda': [0.0, 1.0],
    'learning_rate': [0.1,0.01],
    'n_estimators': [100, 200], 
    'seed': [seed]}
    # GridSearchCV
    score = 'f1_weighted'
    clf = GridSearchCV(svc, param_grid_svm, cv=5, scoring=score, n_jobs = -1)
    CV_rfc = GridSearchCV(rfc, param_grid=param_grid_rfc, cv= 5, scoring=score, n_jobs = -1)
    CV_knn = GridSearchCV(knn, param_grid=param_grid_knn, cv= 5, scoring=score, n_jobs = -1)
    CV_xgb = GridSearchCV(xgb, param_grid=param_grid_xgb, cv= 5, scoring=score, n_jobs = -1)
    # train
    clf.fit(X_train, y_train)
    CV_rfc.fit(X_train, y_train)
    CV_knn.fit(X_train, y_train)
    gnb.fit(X_train,y_train)
    CV_xgb.fit(X_train, y_train)
    # predict
    print("Best parameters set found on development set:")
    print()
    print("Best score: %.1f%%" % (clf.best_score_*100))
    print('SVM:', clf.best_params_)
    print("Best score: %.1f%%" % (CV_rfc.best_score_*100))
    print('Random Forest:', CV_rfc.best_params_)
    print("Best score: %.1f%%" % (CV_knn.best_score_*100))
    print('KNN:', CV_knn.best_params_)
    print("Best score: %.1f%%" % (CV_xgb.best_score_*100))
    print('XGBoost:', CV_xgb.best_params_)
    print()
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print("SVM:")
    y_true_svm, y_pred_svm = y_test, clf.best_estimator_.predict(X_test)
    print(classification_report(y_true_svm, y_pred_svm))
    print()
    print("Random Forest:")
    y_true_rf, y_pred_rf = y_test, CV_rfc.best_estimator_.predict(X_test)
    print(classification_report(y_true_rf, y_pred_rf))
    print()
    print("KNN:")
    y_true_knn, y_pred_knn = y_test, CV_knn.best_estimator_.predict(X_test)
    print(classification_report(y_true_knn, y_pred_knn))
    print()
    print("Gaussian Naive Bayes:")
    #y_true_gnb, y_pred_gnb = y_test, gnb.predict(X_test)
    print(gnb.score(X_test,y_test))
    print()
    print("XGBoost:")
    y_true_xgb, y_pred_xgb = y_test, CV_xgb.best_estimator_.predict(X_test)
    print(classification_report(y_true_xgb, y_pred_xgb))
    print()
    return clf.best_params_, CV_rfc.best_params_, CV_knn.best_params_, CV_xgb.best_params_

In [ ]:
svm_params, rfc_params, knn_params, xgb_params = train_cross_validate_evaluate(X_pca, y, X_test_pca, y_test)

In [ ]:
import numpy as np
num_classes = 3
b = np.array([0,2,1,1,2,0,2,0])
b.reshape(-1) # your initial classes
Y = np.eye(num_classes)[b]
print(b)
print(Y)
print(type(Y))